In [ ]:
from google.colab import drive

drive.mount('/content/drive')
root_dir = "/content/drive/My Drive/"
base_dir = root_dir + 'Tesi/Wiki50/'

WIKI50

In [ ]:
import pandas as pd

wiki_df = pd.read_csv(base_dir + 'wiki50_preprocessed.csv', converters={'result': eval})

In [ ]:
wiki_df

RAKE

In [ ]:
!pip install rake-nltk

In [ ]:
import nltk 
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
from rake_nltk import Rake

r = Rake(min_length=2, max_length=4, language='english') # Uses stopwords for english from NLTK, and all puntuation characters.

In [ ]:
def rake_implement(x,r):
     r.extract_keywords_from_text(x)
     return r.get_ranked_phrases()

wiki_df['rake'] =wiki_df['text'].apply(lambda x: rake_implement(x,r))
print(wiki_df['rake'].head(20))

In [ ]:
wiki_df.head(10)

YAKE

In [ ]:
!pip install git+https://github.com/LIAAD/yake

In [ ]:
import yake

In [ ]:
language = "en"

max_ngram_size = 4
deduplication_thresold = 0.15

In [ ]:
kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold)

In [ ]:
def yake_implement(x):
     keywords = kw_extractor.extract_keywords(x)
     return keywords

wiki_df['yake'] =wiki_df['text'].apply(lambda x: yake_implement(x))

In [ ]:
yake_without_unigrams = []

In [ ]:
for i in range(len(wiki_df)):
  for kw in wiki_df['yake'][i]:
    if(' ' in kw[0]):
        yake_without_unigrams.append(kw[0])
  wiki_df['yake'][i]=yake_without_unigrams
  yake_without_unigrams = []

In [ ]:
yake_lowercase = []

In [ ]:
for i in range(len(wiki_df)):
  for el in wiki_df['yake'][i]:
    el = el.lower()
    yake_lowercase.append(el)
  wiki_df['yake'][i]=yake_lowercase
  yake_lowercase = []

In [ ]:
wiki_df

WORD2PHRASE

In [ ]:
import nltk
nltk.download('punkt')
from textblob import TextBlob
from collections import Counter

In [ ]:
import sys
sys.path.insert(0,'/content/drive/My Drive/Tesi')

In [ ]:
import word2phrase

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import string
import re
import csv

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    return text

wiki_df['nopuncttext'] = wiki_df['text'].apply(lambda x: remove_punct(x))

In [ ]:
wiki_df['tokens']=wiki_df.apply(lambda row: nltk.word_tokenize(row['nopuncttext']), axis=1)

In [ ]:
wiki_df

In [ ]:
df_sentences = wiki_df['tokens'].tolist()

In [ ]:
phrased1 = word2phrase.train_model(df_sentences, min_count=1)
phrased2 = word2phrase.train_model(phrased1, min_count=1)
phrased3 = word2phrase.train_model(phrased2, min_count=1)

list_of_sentences = []

for sentence in phrased3:
     list_of_sentences.append(sentence)

In [ ]:
for i in range(len(list_of_sentences)):
  list_of_sentences[i]= ' '.join(list_of_sentences[i])

In [ ]:
dfW2P = pd.DataFrame(data=list_of_sentences, columns=["text"])

In [ ]:
for i in range(len(dfW2P)):
  dfW2P['text'][i]= dfW2P['text'][i].split()

In [ ]:
dfW2P

In [ ]:
w2p_list = []

In [ ]:
wiki_df['W2P'] = ''

In [ ]:
for i in range(len(dfW2P)):
  for el in dfW2P['text'][i]:
    if '_' in el:
      el = el.replace("_", " ")
      el = el.lower()
      w2p_list.append(el)
  wiki_df['W2P'][i]=w2p_list
  w2p_list = []

In [ ]:
wiki_df = wiki_df.drop(columns=['tokens'])

In [ ]:
wiki_df = wiki_df.drop(columns=['nopuncttext'])

In [ ]:
wiki_df

GENSIM

In [ ]:
import re  # For preprocessing
import pandas as pd  # For data handling
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = stopwords.words('english')

In [ ]:
wiki_df['gensimtext'] = wiki_df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
wiki_df['gensimtext'] = wiki_df['gensimtext'].apply(lambda x: remove_punct(x)) 

In [ ]:
wiki_df['gensimtext']

In [ ]:
from gensim.models.phrases import Phrases, Phraser

In [ ]:
sent = [row.split() for row in wiki_df['gensimtext']]

In [ ]:
bigram = Phrases(sent, min_count=1, threshold=100.0)

trigram  = Phrases(bigram[sent], min_count=1, threshold=100.0)

fourgram  = Phrases(trigram[sent], min_count=1, threshold=100.0)

In [ ]:
bigram_phraser = Phraser(bigram)

In [ ]:
trigram_phraser = Phraser(trigram)

In [ ]:
fourgram_phraser = Phraser(fourgram)

In [ ]:
sentences = bigram_phraser[sent]

In [ ]:
sentences = trigram_phraser[sentences]

In [ ]:
sentences = fourgram_phraser[sentences]

In [ ]:
wordphrases = []

for w in sentences:
  wordphrases.append(w)

In [ ]:
for i in range(len(wordphrases)):
  wordphrases[i]= ' '.join(wordphrases[i])

In [ ]:
dfGensim = pd.DataFrame(data=wordphrases, columns=["text"])

In [ ]:
for i in range(len(dfGensim)):
  dfGensim['text'][i]= dfGensim['text'][i].split()

In [ ]:
dfGensim['text']

In [ ]:
gensim_list = []

In [ ]:
wiki_df['gensim'] = ""

In [ ]:
for i in range(len(dfGensim)):
  for el in dfGensim['text'][i]:
    if '_' in el:
      el = el.replace("_", " ")
      el = el.lower()
      gensim_list.append(el)
  wiki_df['gensim'][i]=gensim_list
  gensim_list = []

In [ ]:
wiki_df = wiki_df.drop(columns=['gensimtext'])

In [ ]:
wiki_df

TOPICRANK

In [ ]:
!pip install git+https://github.com/boudinfl/pke.git

In [ ]:
import string
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import pke

In [ ]:
import spacy
!python -m spacy download en
nlp = spacy.load("en")

In [ ]:
# select the longest sequences of nouns and adjectives, that do
# not contain punctuation marks or stopwords as candidates.
pos = {'NOUN', 'PROPN', 'ADJ'}
stoplist = list(string.punctuation)
stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
stoplist += stopwords.words('english')

In [ ]:
def extract_keyphrases(caption, n):
    extractor = pke.unsupervised.TopicRank()
    extractor.load_document(caption)
    extractor.candidate_selection(pos=pos, stoplist=stoplist)
    extractor.candidate_weighting(threshold=0.5, method='average')
    keyphrases = extractor.get_n_best(n=n, stemming=False)
    return(keyphrases)

In [ ]:
wiki_df['topicrank'] = wiki_df.apply(lambda row: (extract_keyphrases(row['text'],200)),axis=1)

In [ ]:
wiki_df['topicrank'][0]

In [ ]:
topicrank_without_unigrams = []

In [ ]:
for i in range(len(wiki_df)):
  for kw in wiki_df['topicrank'][i]:
    if(' ' in kw[0]):
        topicrank_without_unigrams.append(kw[0])
  wiki_df['topicrank'][i]=topicrank_without_unigrams
  topicrank_without_unigrams = []

In [ ]:
wiki_df['topicrank']

In [ ]:
wiki_df

REMOVE DUPLICATES

In [ ]:
for i in range(len(wiki_df)):
  wiki_df['gensim'][i] = list(dict.fromkeys(wiki_df['gensim'][i]))

In [ ]:
for i in range(len(wiki_df)):
  wiki_df['result'][i] = list(dict.fromkeys(wiki_df['result'][i]))

In [ ]:
for i in range(len(wiki_df)):
  wiki_df['yake'][i] = list(dict.fromkeys(wiki_df['yake'][i]))

In [ ]:
for i in range(len(wiki_df)):
  wiki_df['rake'][i] = list(dict.fromkeys(wiki_df['rake'][i]))

In [ ]:
for i in range(len(wiki_df)):
  wiki_df['W2P'][i] = list(dict.fromkeys(wiki_df['W2P'][i]))

In [ ]:
for i in range(len(wiki_df)):
  wiki_df['topicrank'][i] = list(dict.fromkeys(wiki_df['topicrank'][i]))

In [ ]:
wiki_df

In [ ]:
# if you want to save dataframe

compression_opts = dict(method='zip', archive_name='wiki50_mwe.csv')

wiki_df.to_csv('wiki50_mwe.zip', index=False, compression=compression_opts)

PRECISION & RECALL

In [ ]:
matches_df = pd.DataFrame(columns=['rakeMatches']) 

In [ ]:
matches_df["rakeMatches"] = [set(wiki_df.loc[r, "result"]) & set(wiki_df.loc[r, "rake"]) for r in range(len(wiki_df))]

In [ ]:
matches_df["yakeMatches"] = [set(wiki_df.loc[r, "result"]) & set(wiki_df.loc[r, "yake"]) for r in range(len(wiki_df))]

In [ ]:
matches_df["topicrankMatches"] = [set(wiki_df.loc[r, "result"]) & set(wiki_df.loc[r, "topicrank"]) for r in range(len(wiki_df))]

In [ ]:
matches_df["gensimMatches"] = [set(wiki_df.loc[r, "result"]) & set(wiki_df.loc[r, "gensim"]) for r in range(len(wiki_df))]

In [ ]:
matches_df["w2pMatches"] = [set(wiki_df.loc[r, "result"]) & set(wiki_df.loc[r, "W2P"]) for r in range(len(wiki_df))]

In [ ]:
matches_df

In [ ]:
result_list = wiki_df['result'].tolist()

In [ ]:
count_result = []

for i in range(len(result_list)):
  for el in result_list[i]:
    count_result.append(el)

In [ ]:
len(count_result)

YAKE PRECISION & RECALL

In [ ]:
yakeMatch_list = list(map(list, matches_df['yakeMatches']))

In [ ]:
count_yakeMatch = []

for i in range(len(yakeMatch_list)):
  for el in yakeMatch_list[i]:
    count_yakeMatch.append(el)

In [ ]:
yake_list = wiki_df['yake'].tolist()

In [ ]:
count_yake = []

for i in range(len(yake_list)):
  for el in yake_list[i]:
    count_yake.append(el)

In [ ]:
recall_yake = len(count_yakeMatch)/len(count_result) #recall

In [ ]:
precision_yake = len(count_yakeMatch)/len(count_yake) #precision

In [ ]:
fscore_yake = 2 * (precision_yake * recall_yake)/(precision_yake + recall_yake)

In [ ]:
recall_yake

In [ ]:
precision_yake

In [ ]:
fscore_yake

RAKE PRECISION & RECALL

In [ ]:
rakeMatch_list = list(map(list, matches_df['rakeMatches']))

In [ ]:
count_rakeMatch = []

for i in range(len(rakeMatch_list)):
  for el in rakeMatch_list[i]:
    count_rakeMatch.append(el)

In [ ]:
rake_list = wiki_df['rake'].tolist()

In [ ]:
count_rake = []

for i in range(len(rake_list)):
  for el in rake_list[i]:
    count_rake.append(el)

In [ ]:
recall_rake = len(count_rakeMatch)/len(count_result) #recall

In [ ]:
precision_rake = len(count_rakeMatch)/len(count_rake)

In [ ]:
fscore_rake = 2 * (precision_rake * recall_rake)/(precision_rake + recall_rake)

In [ ]:
recall_rake

In [ ]:
precision_rake

In [ ]:
fscore_rake

TOPICRANK PRECISION & RECALL

In [ ]:
trMatch_list = list(map(list, matches_df['topicrankMatches']))

In [ ]:
count_trMatch = []

for i in range(len(trMatch_list)):
  for el in trMatch_list[i]:
    count_trMatch.append(el)

In [ ]:
tr_list = wiki_df['topicrank'].tolist()

In [ ]:
count_tr = []

for i in range(len(tr_list)):
  for el in tr_list[i]:
    count_tr.append(el)

In [ ]:
recall_t = len(count_trMatch)/len(count_result) #recall

In [ ]:
precision_t = len(count_trMatch)/len(count_tr)

In [ ]:
fscore_t = 2 * (precision_t * recall_t)/(precision_t + recall_t)

In [ ]:
recall_t

In [ ]:
precision_t 

In [ ]:
fscore_t

GENSIM PRECISION & RECALL

In [ ]:
gensimMatch_list = list(map(list, matches_df['gensimMatches']))

In [ ]:
count_gensimMatch = []

for i in range(len(gensimMatch_list)):
  for el in gensimMatch_list[i]:
    count_gensimMatch.append(el)

In [ ]:
gensim_list = wiki_df['gensim'].tolist()

In [ ]:
count_gensim = []

for i in range(len(gensim_list)):
  for el in gensim_list[i]:
    count_gensim.append(el)

In [ ]:
recall_g = len(count_gensimMatch)/len(count_result) #recall

In [ ]:
precision_g = len(count_gensimMatch)/len(count_gensim)

In [ ]:
fscore_g = 2 * (precision_g * recall_g)/(precision_g + recall_g)

In [ ]:
recall_g

In [ ]:
precision_g

In [ ]:
fscore_g

W2P PRECISION & RECALL

In [ ]:
w2pMatch_list = list(map(list, matches_df['w2pMatches']))

In [ ]:
count_w2pMatch = []

for i in range(len(w2pMatch_list)):
  for el in w2pMatch_list[i]:
    count_w2pMatch.append(el)

In [ ]:
w2p_list = wiki_df['W2P'].tolist()

In [ ]:
count_w2p = []

for i in range(len(w2p_list)):
  for el in w2p_list[i]:
    count_w2p.append(el)

In [ ]:
recall_w = len(count_w2pMatch)/len(count_result) #recall

In [ ]:
precision_w = len(count_w2pMatch)/len(count_w2p)

In [ ]:
fscore_w = 2 * (precision_w * recall_w)/(precision_w + recall_w)

In [ ]:
recall_w

In [ ]:
precision_w

In [ ]:
fscore_w